# Apriori算法进行关联分析

## 基本概念
### 关联分析
从大规模数据集中寻找物品间的隐含关系，或称关联规则学习（association rule learning）
### 频繁项集（frequent item sets）
经常出现在一块的物品的集合
### 关联规则
暗示两种物品间可能存在很强的关系
### 支持度（support）
一个项集的支持度被定义为数据集中包含该项集的记录所占的比例
support({A,B}) = num(A∪B) / W = P(A∩B)
num(A∪B)表示含有物品集{A,B}的事务集的个数，不是数学中的并集。
支持度是针对项集来说的，因此可以定义一个最小支持度，而只保留满足最小支持度的项集
### 置信度（confidence）
置信度(confidence)揭示了A出现时B是否一定出现，如果出现，则出现的概率是多大。
Confidence(A->B) = support({A,B}) / support({A}) = P(B|A)
支持度和置信度是用来量化关联分析是否成功的方法

### Apriori原理
如果某个项集是频繁的，那么它的所有子集也是频繁的，其逆否命题：
如果一个项集是非频繁集，则它的所有超集都是非频繁的
作用：减少关联规则学习时所需的计算量

扫描数据集生成候选项集伪代码如下
对数据集中的每条交易记录tran：
    检查一下can是否tran的子集：
    是则增加can的计数值
对每个候选项集：
    如果其支持度不低于最小值，则保留该项集
    返回所有频繁项列表

In [1]:
from numpy import *
def loadDataSet():                                                                         # 用于测试的简单数据集
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]


In [2]:
def createC1(dataSet):                                                                     # 构建大小为1的所有候选集的集合
    C1 = []
    for transaction in dataSet:                                                            # 遍历数据集中的每条记录
        for item in transaction:                                                           # 遍历每条记录中的每个物品
            if not [item] in C1:                                                           
                C1.append([item])                                                          # C1中还不存在的情况下，加入C1中
    C1.sort()                                                                              # 排序
    return list(map(frozenset, C1)) # use frozen set so we can use it as a key in a dict   # 返回 对C1中每个项构建一个不变集合    只包含一个物品项的列表，python不能创建只有一个整数的集合 
                                    # frozenset类型与set类型均不能被用户所修改，但前者可以将这些集合作为字典的key，后者不能

In [3]:
def scanD(D, Ck, minSupport):                                                           # 该函数用于从候选项集Ck生成满足最小支持度的频繁项Lk
    '''input:  D                数据集的每一条记录的集合
               Ck               候选项集列表Ck
               minSupport       最小支持度
        output：retList          最频繁项集Lk（dict，key为项集组合，value为计数）
                supportData      Lk的项集的支持度（dict，key为项集组合，value为其对应的支持度）
    '''                                                                                 # 计数
    ssCnt = {}                                                                          # 创建空字典
    for tid in D:                                                                       # 遍历每一条交易记录
        for can in Ck:                                                                  # 遍历候选集Ck
            if can.issubset(tid):                                                       # 如果Ck中的集合是记录的一部分
                                                                                        # 增加字典中候选集key对应的计数值
                if can not in ssCnt: ssCnt[can]=1                                       # 情况1： 第一次记录
                else: ssCnt[can] += 1                                                   # 情况2：不是第一次
                                                                                        # 遍历完成得到字典ssCnt，key为Ck项集组合，value为计数值
                                                                                        # 当扫描完数据集中所有项以及所有候选集时，得到key为候选项，value为其计数的字典ssCnt
                                                                                       
                                                                                        # 计算支持度，筛掉不满足支持度的项
    numItems = float(len(D))                                                            # 总交易记录数         
    retList = []                                                                        # 初始化Lk
    supportData = {}                                                                    # 初始化Lk中每一个项的支持度
    for key in ssCnt:                                                                   # 遍历字典
        support = ssCnt[key]/numItems                                                   # 计算支持度
        if support >= minSupport:                                                       # 若支持度大于设定的最小支持度
            retList.insert(0,key)                                                       # 加入relist字典
        supportData[key] = support                                                      
    return retList, supportData                                                         # 返回最频繁项集Lk和支持度

In [4]:
#尝试
dataSet = loadDataSet()
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [5]:
D = list(map(set, dataSet)) # 将数据集的每一条交易记录变为集合
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [11]:
L1, suppData0 = scanD(D, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [12]:
suppData0

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

### 组织完整的Apriori算法
伪代码如下
当集合中项的个数大于0时：
    构建一个k个项组成的候选项集的列表
    检查数据确认每个项集都是频繁的
    保留频繁项集并构建k+1项组成的候选项集的列表

In [6]:
def aprioriGen(Lk, k):                                                         # 由最频繁集Lk创建其超集Ck+1
    '''input:  Lk              频繁项集列表
               k               生成的项集的项的元素个数
       output：retList         创建下一轮的候选集Ck+1
    '''
    retList = []                                                              # 初始化超集列表
    lenLk = len(Lk)                                                           # 计算Lk中项的个数
    for i in range(lenLk):                                                    # Lk中的频繁项集进行两两比较
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]                                            # 比较两个项的前k-2个元素，若相等即可合并成新的项，不满足则有两种情况1）已考虑过的超集  2）其子集非频繁（）Apriori原理的逆否命题
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2:                                                        # if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j])                                 # set union  并加入超集列表                    
    return retList                                                            # 返回超集列表                                            
# 举例， 以{0}{1}{2}作为输入，会生成{0,1}{0,2}{1,2}
# k-2的原因，{0,1}{0,2}{1,2}生成三元素集合{0,1,2}仅进行一次操作

In [16]:
def apriori(dataSet, minSupport = 0.5):                                      # Apriori 算法
    '''input:  dataSet               数据集
               minSupport = 0.5      最小支持度
       output：retList                Ck+1
    '''
    C1 = createC1(dataSet)                                                   # 数据集创建C1
    D = list(map(set, dataSet))                                              # 数据集记录生成记录集合列表
    L1, supportData = scanD(D, C1, minSupport)                               # scanD生成L1及对应的支持度
    L = [L1]                                                                 # 初始化总的最频繁集L
    k = 2
    while (len(L[k-2]) > 0):                                                 # 有了L1，继续寻找L2,3....直至下一个大的项集为空
        Ck = aprioriGen(L[k-2], k)                                           # aprioriGen 由Lk生成候选集Ck+1
        Lk, supK = scanD(D, Ck, minSupport)                                  # scan DB to get Lk，丢掉不满足最小支持度要求的项，  由Ck+1生成Lk+1   
        supportData.update(supK)                                             # 更新支持度 （ update() 函数把字典dict2的键/值对更新到dict里 ）                   
        L.append(Lk)                                                         # 总的最频繁项集L中加入Lk+1
        k += 1                                                               
    return L, supportData

In [17]:
L, suppData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [18]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [19]:
# 每个项集都是在apriori()中调用函数aprioriGen()来生成的
# aprioriGen的工作流程
aprioriGen(L[1],3)
# 因为{1，2}{1，3}{1，5}均为非频繁

[frozenset({2, 3, 5})]

In [20]:
L, suppData = apriori(dataSet, minSupport=0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

### 从频繁项集中挖掘关联规则

In [28]:
def generateRules(L, supportData, minConf=0.7):                                          # supportData is a dict coming from scanD
    '''input  L                   频繁项集列表L
              supportData         包含那些频繁项集支持数据的字典 （每个项的支持度）
              minConf=0.7         最小可行度阈值
       output bigRuleList         包含可信度的规则列表
    '''
    bigRuleList = []                                                                     # 初始化关联规则列表
    for i in range(1, len(L)):                                                           # only get the sets with two or more items
        for freqSet in L[i]:                                                             # 遍历L[i]中的每一项
            H1 = [frozenset([item]) for item in freqSet]                                 # 遍历L[i]每一项中的每一元素，（作为rulesFromConseq或rulesFromConseq的出现在规则右部的元素列表H）
            if (i > 1):                                                                  # L[2]及其2以上的每一项中的元素个数大于2，需要先生成候选规则集
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)          # 调用rulesFromConseq，生成候选规则集合
                
            else:                                                                        # L[1]中每一项仅有两个元素，不需要生成规则，可以直接计算可信度并评价
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)                 # 调用calcConf，对规则进行评价
    return bigRuleList         

In [29]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):                                 # 对生成的规则进行评估
    '''input  freqSet           L[i]中的每一项
              H                 出现在规则右部的元素列表H
              supportData       含那些频繁项集支持数据的字典 （每个项的支持度）
              brl               包含可信度的规则列表
              minConf=0.7       最小可信度
       output prunedH           满足最小可信度的规则列表
    '''
    prunedH = []                                                                       # create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]                        # calc confidence
        if conf >= minConf: 
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)                                                    # 满足最小可信度的规则保存在 prunedH中
    return prunedH


In [30]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):                        # 生成候选规则集合
    '''input  freqSet           L[i]中的每一项
              H                 出现在规则右部的元素列表H
              supportData       含那些频繁项集支持数据的字典 （每个项的支持度）
              brl               包含可信度的规则列表
              minConf=0.7       最小可信度
       output rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)               递归
    '''
    m = len(H[0])
    if (len(freqSet) > (m + 1)):                                                     # try further merging  如果L[i]中的每一项的元素个数大于2，则出现在规则右部的元素列表H需要进一步考虑合并元素
        Hmp1 = aprioriGen(H, m+1)                                                    # create Hm+1 new candidates 对H寻找超集
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)                    # 评估筛选满足最小可信度的规则列表
        if (len(Hmp1) > 1):                                                          # 如果超集还可以合并，则继续递归
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [31]:
L, suppData = apriori(dataSet, minSupport=0.5)

In [32]:
rules = generateRules(L, suppData, minConf=0.7)
rules

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

### 示例：发现毒蘑菇的相似特征

In [34]:
mushDatSet = [line.split() for line in open("mushroom.dat").readlines()]
mushDatSet[5]

['2',
 '3',
 '10',
 '14',
 '23',
 '26',
 '34',
 '36',
 '39',
 '41',
 '52',
 '55',
 '59',
 '63',
 '67',
 '76',
 '85',
 '86',
 '90',
 '93',
 '98',
 '108',
 '114']

In [36]:
import numpy as np
np.shape(mushDatSet)

(8124, 23)

In [37]:
L, suppData = apriori(mushDatSet, minSupport=0.3)

In [ ]:
L

In [40]:
# 在结果中搜索包含有毒特征值2的频繁项集
for item in L[2]:
    if item.intersection('2'):
        print(item)

frozenset({'39', '2', '53'})
frozenset({'2', '90', '53'})
frozenset({'2', '85', '53'})
frozenset({'2', '34', '53'})
frozenset({'86', '2', '53'})
frozenset({'28', '2', '53'})
frozenset({'28', '39', '2'})
frozenset({'28', '2', '90'})
frozenset({'86', '28', '2'})
frozenset({'28', '2', '85'})
frozenset({'28', '63', '2'})
frozenset({'59', '28', '2'})
frozenset({'28', '2', '34'})
frozenset({'39', '2', '93'})
frozenset({'39', '2', '90'})
frozenset({'39', '2', '85'})
frozenset({'39', '2', '76'})
frozenset({'39', '2', '67'})
frozenset({'39', '63', '2'})
frozenset({'39', '36', '2'})
frozenset({'39', '2', '34'})
frozenset({'2', '90', '93'})
frozenset({'86', '2', '93'})
frozenset({'86', '2', '90'})
frozenset({'86', '2', '85'})
frozenset({'86', '76', '2'})
frozenset({'86', '2', '67'})
frozenset({'86', '63', '2'})
frozenset({'86', '36', '2'})
frozenset({'86', '2', '34'})
frozenset({'86', '23', '2'})
frozenset({'86', '39', '2'})
frozenset({'2', '85', '93'})
frozenset({'90', '2', '85'})
frozenset({'76

In [39]:
for item in L[3]:
    if item.intersection('2'):
        print(item)

frozenset({'59', '28', '2', '39'})
frozenset({'59', '28', '2', '34'})
frozenset({'59', '28', '2', '63'})
frozenset({'59', '28', '2', '85'})
frozenset({'59', '28', '2', '86'})
frozenset({'59', '28', '90', '2'})
frozenset({'28', '2', '63', '34'})
frozenset({'28', '2', '85', '63'})
frozenset({'28', '2', '85', '34'})
frozenset({'28', '2', '86', '53'})
frozenset({'28', '2', '86', '39'})
frozenset({'28', '2', '86', '34'})
frozenset({'28', '2', '86', '63'})
frozenset({'28', '2', '85', '86'})
frozenset({'28', '90', '2', '86'})
frozenset({'28', '2', '90', '34'})
frozenset({'28', '2', '90', '85'})
frozenset({'28', '2', '39', '34'})
frozenset({'28', '2', '39', '63'})
frozenset({'28', '2', '85', '39'})
frozenset({'28', '2', '90', '39'})
frozenset({'28', '2', '34', '53'})
frozenset({'28', '2', '85', '53'})
frozenset({'28', '2', '90', '53'})
frozenset({'28', '2', '39', '53'})
frozenset({'2', '86', '34', '53'})
frozenset({'2', '85', '86', '53'})
frozenset({'2', '90', '86', '53'})
frozenset({'2', '86'